In [1]:
import cv2

In [2]:
import numpy as np
import pandas as pd
import math

In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

# Implementing face and face part detection, getting some intuitions how they behave

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default_.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye_.xml')
l_eye_cascade = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
l_eye_cascade2 = cv2.CascadeClassifier('left_eye_modesto_train.xml')
oneeye_cascade = cv2.CascadeClassifier('haarcascade_one_eye.xml')
frontaleyes_cascade = cv2.CascadeClassifier('frontalEyes35x16.xml')
mouth_cascade = cv2.CascadeClassifier('Mouth.xml')
nose_cascade = cv2.CascadeClassifier('Nariz.xml')

In [134]:
# folder for first testing
mypath= 'early_test'

In [135]:
import os
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [136]:
print mypath+'/'+ onlyfiles[1]
img = cv2.imread(mypath+'/'+ onlyfiles[1])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

early_test/19_Couple_Couple_19_429.jpg


In [137]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [138]:
def detect_in_dir(filenames,cascade,output_folder,color,scalef,min_neighbors,minsize):
    if not os.path.exists(mypath+'/'+output_folder):
        os.makedirs(mypath+'/'+output_folder)
    for imagename in filenames:
        img = cv2.imread(mypath+'/'+ imagename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = cascade.detectMultiScale(gray, scaleFactor=scalef, minNeighbors=min_neighbors,minSize=minsize)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
        cv2.imwrite(mypath+'/'+ output_folder+"/"+imagename,img)
    return None

In [139]:
def detect_in_dir_fullmode(mypath,isface_lst,filenames,cascade,output_folder,color,scalef,min_neighbors,minsize):
    if not os.path.exists(mypath+'/'+output_folder):
        os.makedirs(mypath+'/'+output_folder)
    i=0
    predicts=[]
    for imagename in filenames:
        img = cv2.imread(mypath+'/'+ imagename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = cascade.detectMultiScale(gray, scaleFactor=scalef, minNeighbors=min_neighbors,minSize=minsize)
        if len(faces)>0: 
            predicts.append(1)
            pred=1
        else: 
            predicts.append(0)
            pred=0
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]

        cv2.imwrite(mypath+'/'+ output_folder+"/"+'isface'+str(isface_lst[i])+'_isfacedetected'+str(pred)+'_'+ str.split(imagename,'.')[0]+'.jpg',img)
        i=i+1
    return predicts

In [ ]:
# you may not have to run these cells below, the results are in the output folders

In [140]:
detect_in_dir(filenames=onlyfiles,cascade=face_cascade,output_folder='results_face_test',color=(255,0,0),
              scalef=1.3,min_neighbors=5,minsize=(30,30))

In [141]:
detect_in_dir(filenames=onlyfiles,cascade=oneeye_cascade,output_folder='results_oneeye_minN6',color=(0,255,0),
              scalef=1.2,min_neighbors=6,minsize=(20,20))

In [147]:
detect_in_dir(filenames=onlyfiles,cascade=frontaleyes_cascade,output_folder='results_frontaleyes_minN6',color=(0,255,0),
              scalef=1.2,min_neighbors=6,minsize=(20,20))

In [143]:
detect_in_dir(filenames=onlyfiles,cascade=mouth_cascade,output_folder='results_mouth_minn100',color=(0,255,0),
              scalef=1.2,min_neighbors=100,minsize=(20,20))

In [144]:
detect_in_dir(filenames=onlyfiles,cascade=nose_cascade,output_folder='results_nose_minn6',color=(0,0,255),
              scalef=1.2,min_neighbors=6,minsize=(20,20))

In [145]:
detect_in_dir(filenames=onlyfiles,cascade=eye_cascade,output_folder='results_eye_default_minN5',color=(0,255,0),
              scalef=1.3,min_neighbors=5,minsize=(20,20))

# Face detection on test dataset

In [16]:
def perf_measure(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]

    return(TP, FP, TN, FN)

### Reading filenames of train and test sets

In [17]:
faces_train_path= 'faces_train'
nonfaces_train_path= 'non_faces_train'
faces_test_path= 'faces_test'
nonfaces_test_path= 'non_faces_test'

In [18]:
faces_train_files = [f for f in listdir(faces_train_path) if isfile(join(faces_train_path, f))]
faces_test_files = [f for f in listdir(faces_test_path) if isfile(join(faces_test_path, f))]
nonfaces_train_files = [f for f in listdir(nonfaces_train_path) if isfile(join(nonfaces_train_path, f))]
nonfaces_test_files = [f for f in listdir(nonfaces_test_path) if isfile(join(nonfaces_test_path, f))]

In [19]:
print faces_train_files[0]
print faces_test_files[0]
print nonfaces_train_files[0]
print nonfaces_test_files[0]

BioID_0000.pgm
BioID_0004.pgm
0100.jpg
0107.jpg


#importing to pandas

In [20]:
faces_train=pd.DataFrame(faces_train_files)
faces_train.rename(columns = {0:'filename'}, inplace = True)
faces_train['face']=1
faces_test=pd.DataFrame(faces_test_files)
faces_test.rename(columns = {0:'filename'}, inplace = True)
faces_test['face']=1
nonfaces_train=pd.DataFrame(nonfaces_train_files)
nonfaces_train.rename(columns = {0:'filename'}, inplace = True)
nonfaces_train['face']=0
nonfaces_test=pd.DataFrame(nonfaces_test_files)
nonfaces_test.rename(columns = {0:'filename'}, inplace = True)
nonfaces_test['face']=0

### Running face detection on Test dataset with recommended parameters
(scalef=1.3,min_neighbors=5)

In [97]:
y_pred_isface1=detect_in_dir_fullmode(mypath='faces_test', isface_lst=faces_test['face'] ,filenames=faces_test_files ,cascade=face_cascade,
                                      output_folder='results_face_test',color=(255,0,0),scalef=1.3,min_neighbors=5,minsize=(30,30))

In [98]:
y_pred_isface0=detect_in_dir_fullmode(mypath='non_faces_test', isface_lst=nonfaces_test['face'] ,filenames=nonfaces_test_files ,cascade=face_cascade,
                                      output_folder='results_face_test',color=(255,0,0),scalef=1.3,min_neighbors=5,minsize=(30,30))

In [99]:
y_pred=y_pred_isface1+y_pred_isface0
y_pred_arr=np.array(y_pred)
y_true_arr=np.concatenate((faces_test['face'].values, nonfaces_test['face'].values))
print len(y_pred_arr)
print len(y_true_arr)

687
687


### Results of face detection

In [100]:
perfs=perf_measure(y_true_arr,y_pred_arr)
print 'True positives: ',perfs[0]
print 'False positives: ',perfs[1]
print 'True negatives: ',perfs[2]
print 'False negatives: ',perfs[3]
print 'accuracy: ',accuracy_score(y_true_arr,y_pred_arr)
print 'precision: ',precision_score(y_true_arr,y_pred_arr)
print 'recall: ',recall_score(y_true_arr,y_pred_arr)

True positives:  418
False positives:  13
True negatives:  193
False negatives:  63
accuracy:  0.889374090247
precision:  0.969837587007
recall:  0.869022869023


# Implementing new face detection from features based on face-part detection

### Feature creation from face part detection in order to make use of them in face detection

In [23]:
pd.options.mode.chained_assignment = None  # default='warn'

In [254]:
def detect_eyes_mouth_making_features(mypath,isface_lst,filenames,output_folder,df):
    if not os.path.exists(mypath+'/'+output_folder):
        os.makedirs(mypath+'/'+output_folder)
    i=0
    #predicts=[]
    df['eyes_number_of']=0           
    df['eyes_rect_mean']=0.0
    df['eyes_rect_min']=0.0
    df['eyes_rect_max']=0.0
    df['eyes_rect_median']=0.0
    df['eyes_placement_characteristip']=0.0
    
    df['mouth_number_of']=0                     
    df['mouth_rect_mean']=0.0
    df['mouth_rect_min']=0.0
    df['mouth_rect_max']=0.0
    df['mouth_rect_median']=0.0
    df['mouth_placement_characteristip']=0.0

    df['mouth2_number_of']=0
    df['mouth2_rect_mean']=0.0
    df['mouth2_rect_min']=0.0
    df['mouth2_rect_max']=0.0
    df['mouth2_rect_median']=0.0

    df['mouth_eyes_dist']=0.0
    df['mouth2_eyes_dist']=0.0
        
    coloreye=(0,255,0)
    colormouth=(0,0,255)
    colormouth2=(0,255,255)
    for imagename in df['filename']:
        img = cv2.imread(mypath+'/'+ imagename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3,minNeighbors=5,minSize=(20,20))
        mouths=mouth_cascade.detectMultiScale(gray, scaleFactor=1.2,minNeighbors=10,minSize=(20,20))
        mouths2=mouth_cascade.detectMultiScale(gray, scaleFactor=1.2,minNeighbors=100,minSize=(20,20))
                
        rect_temp_eyes=[]
        x_temp_eyes=[]
        y_temp_eyes=[]
        
        rect_temp_mouth=[]
        x_temp_mouth=[]
        y_temp_mouth=[]
        
        rect_temp_mouth2=[]
        x_temp_mouth2=[]
        y_temp_mouth2=[]
        
        for (x,y,w,h) in eyes:
            cv2.rectangle(img,(x,y),(x+w,y+h),coloreye,2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            rect_temp_eyes.append(math.hypot(w, h)) #diagonal of rectangles              
            x_temp_eyes.append(x+(w/2))
            y_temp_eyes.append(y+(h/2))
        eyes_center_x=np.mean(x_temp_eyes)
        eyes_center_y=np.mean(y_temp_eyes)
        
        for (xm,ym,wm,hm) in mouths:
            cv2.rectangle(img,(xm,ym),(xm+wm,ym+hm),colormouth,3)
            roi_gray_m = gray[ym:ym+hm, xm:xm+wm]
            roi_color_m = img[ym:ym+hm, xm:xm+wm]
            rect_temp_mouth.append(math.hypot(wm, hm)) #diagonal of rectangles              
            x_temp_mouth.append(xm+(wm/2))
            y_temp_mouth.append(ym+(hm/2))
        mouth_center_x=np.mean(x_temp_mouth)
        mouth_center_y=np.mean(y_temp_mouth)
        
        for (xm2,ym2,wm2,hm2) in mouths2:
            cv2.rectangle(img,(xm2,ym2),(xm2+wm2,ym2+hm2),colormouth2,2)
            roi_gray_m2 = gray[ym2:ym2+hm2, xm2:xm2+wm2]
            roi_color_m2 = img[ym2:ym2+hm2, xm2:xm2+wm2]
            rect_temp_mouth2.append(math.hypot(wm2, hm2)) #diagonal of rectangles              
            x_temp_mouth2.append(xm2+(wm2/2))
            y_temp_mouth2.append(ym2+(hm2/2))
        mouth2_center_x=np.mean(x_temp_mouth2)
        mouth2_center_y=np.mean(y_temp_mouth2)
        
        if len(eyes)>0:
            df['eyes_number_of'][i]=len(eyes)                     
            df['eyes_rect_mean'][i]=np.mean(rect_temp_eyes)
            df['eyes_rect_min'][i]=np.min(rect_temp_eyes)
            df['eyes_rect_max'][i]=np.max(rect_temp_eyes)
            df['eyes_rect_median'][i]=np.median(rect_temp_eyes)
            df['eyes_placement_characteristip'][i]=((np.std(x_temp_eyes)+np.std(y_temp_eyes))/2) / np.median(rect_temp_eyes)
        if len(mouths)>0:
            df['mouth_number_of'][i]=len(mouths)                     
            df['mouth_rect_mean'][i]=np.mean(rect_temp_mouth)
            df['mouth_rect_min'][i]=np.min(rect_temp_mouth)
            df['mouth_rect_max'][i]=np.max(rect_temp_mouth)
            df['mouth_rect_median'][i]=np.median(rect_temp_mouth)
            df['mouth_placement_characteristip'][i]=((np.std(x_temp_mouth)+np.std(y_temp_mouth))/2)/np.median(rect_temp_mouth)
        if len(mouths2)>0:
            df['mouth2_number_of'][i]=len(mouths2)                     
            df['mouth2_rect_mean'][i]=np.mean(rect_temp_mouth2)
            df['mouth2_rect_min'][i]=np.min(rect_temp_mouth2)
            df['mouth2_rect_max'][i]=np.max(rect_temp_mouth2)
            df['mouth2_rect_median'][i]=np.median(rect_temp_mouth2)
                
        #distances related to rectangles
        if len(mouths)>0 and len(eyes)>0:
            df['mouth_eyes_dist'][i]=math.hypot(mouth_center_x-eyes_center_x, mouth_center_y-eyes_center_y) / np.median(rect_temp_eyes)
        if len(mouths2)>0 and len(eyes)>0:
            df['mouth2_eyes_dist'][i]=math.hypot(mouth2_center_x-eyes_center_x, mouth2_center_y-eyes_center_y) / np.median(rect_temp_mouth2)
        
        cv2.imwrite(mypath+'/'+ output_folder+"/"+'isface'+str(isface_lst[i]) + '_'+ str.split(imagename,'.')[0]+'.jpg',img)
        i=i+1
    return None

### Testing on a small dataset

In [255]:
####### testing dataframe creation on a small test set ########

early_test_small_path= 'early_test_small'
early_test_small_files = [f for f in listdir(proba_small_path) if isfile(join(proba_small_path, f))]
early_test_small=pd.DataFrame(early_test_small_files)
early_test_small.rename(columns = {0:'filename'}, inplace = True)
early_test_small['face']=1

In [256]:
detect_eyes_mouth_making_features(mypath='early_test_small',isface_lst=early_test_small['face'],filenames=early_test_small['filename'],output_folder='results',df=early_test_small)

In [257]:
early_test_small

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_rect_max,mouth_rect_median,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist
0,0115.jpg,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,6,56.738039,...,76.661594,51.363973,1.024274,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brad-Pitt.jpg,1,2,60.811183,53.740115,67.882251,60.811183,0.332998,4,96.982857,...,128.280942,96.640156,0.456494,1,94.154129,94.154129,94.154129,94.154129,0.115000,1.104572
2,merge_male1.jpg,1,2,70.710678,69.296465,72.124892,70.710678,0.304056,5,110.891256,...,193.793705,96.896852,0.522950,1,104.957134,104.957134,104.957134,104.957134,0.143534,1.043381
3,sebastian_petrovski_girl_two_light.jpg,1,3,101.823376,42.426407,134.350288,128.693434,0.544898,4,122.879158,...,151.604749,136.770854,0.752826,1,151.604749,151.604749,151.604749,151.604749,0.432560,0.896468


### Creating features for train dataset

In [258]:
face_nonface_train = pd.concat([faces_train, nonfaces_train], ignore_index=True)

In [259]:
detect_eyes_mouth_making_features(mypath='faces_nonfaces_train',isface_lst=face_nonface_train['face'],filenames=face_nonface_train['filename'],output_folder='results_eye_mouth10_100',df=face_nonface_train)

In [260]:
face_nonface_train.head()

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_rect_max,mouth_rect_median,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist
0,BioID_0000.pgm,1,2,63.639610,60.811183,66.468037,63.639610,0.561757,5,68.188297,...,77.175126,73.573093,0.578377,1,77.175126,77.175126,77.175126,77.175126,1.937057,1.735404
1,BioID_0001.pgm,1,4,53.386562,36.769553,62.225397,57.275649,0.702352,4,75.674406,...,99.984999,68.171225,0.565285,1,67.742158,67.742158,67.742158,67.742158,1.082610,1.375036
2,BioID_0002.pgm,1,1,57.982756,57.982756,57.982756,57.982756,0.000000,4,90.379927,...,124.679589,84.548980,0.375611,1,72.201108,72.201108,72.201108,72.201108,2.520650,2.058387
3,BioID_0003.pgm,1,1,62.225397,62.225397,62.225397,62.225397,0.000000,4,80.390720,...,107.186753,71.772672,0.533174,1,71.344236,71.344236,71.344236,71.344236,2.279030,2.219068
4,BioID_0005.pgm,1,3,53.268711,46.669048,59.396970,53.740115,0.751227,4,79.275758,...,99.984999,73.573305,0.551335,1,71.344236,71.344236,71.344236,71.344236,1.355474,1.404518


In [261]:
y_train=face_nonface_train['face'].values
y_train

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [262]:
X_train=face_nonface_train.iloc[:,2:].values

### Creating features for test dataset

In [263]:
face_nonface_test = pd.concat([faces_test, nonfaces_test], ignore_index=True)

In [264]:
detect_eyes_mouth_making_features(mypath='faces_nonfaces_test',isface_lst=face_nonface_test['face'],filenames=face_nonface_test['filename'],output_folder='results_eye_mouth10_100',df=face_nonface_test)

In [265]:
face_nonface_test.head()

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_rect_max,mouth_rect_median,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist
0,BioID_0004.pgm,1,1,62.225397,62.225397,62.225397,62.225397,0.000000,4,79.275981,...,101.355809,72.458664,0.546263,1,70.830784,70.830784,70.830784,70.830784,2.442300,2.408015
1,BioID_0024.pgm,1,2,59.396970,57.982756,60.811183,59.396970,0.277792,4,69.286029,...,79.404030,71.515884,0.424998,1,79.404030,79.404030,79.404030,79.404030,0.621931,1.146193
2,BioID_0034.pgm,1,2,50.204581,46.669048,53.740115,50.204581,0.293798,4,70.872212,...,88.836929,72.201271,0.384075,1,80.262071,80.262071,80.262071,80.262071,1.098233,0.857526
3,BioID_0040.pgm,1,1,57.982756,57.982756,57.982756,57.982756,0.000000,5,81.907908,...,123.822454,73.573093,0.423817,1,73.573093,73.573093,73.573093,73.573093,0.249211,0.496105
4,BioID_0047.pgm,1,2,56.568542,55.154329,57.982756,56.568542,0.278423,4,75.588248,...,91.065910,70.657829,0.623334,1,69.971423,69.971423,69.971423,69.971423,0.972475,0.993674


In [266]:
y_test=face_nonface_test['face'].values

In [267]:
X_test=face_nonface_test.iloc[:,2:].values

### Training and testing Face detection by facepart detection
Creating and tuning classiefier in order to detect faces.
Features for classiefier were created by face part detection (haar cascades, openCV, Viola-Jones)

In [40]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectKBest

In [44]:
from sklearn.preprocessing import StandardScaler

In [270]:
#clf=RandomForestClassifier(max_depth=8, n_estimators=300, min_samples_leaf=5,class_weight='balanced')
#clf=SVC(kernel='linear', C=1000, gamma=0.0001, )#class_weight='balanced'
clf = Pipeline([
    ('tr',StandardScaler()),
    ('clf_',  SVC(kernel='rbf', C=5000, gamma=0.00002,class_weight='balanced' ))
])

clf.fit(X_train, y_train)
y_pred_faceparts = clf.predict(X_test)

In [271]:
perfs2=perf_measure(y_test,y_pred_faceparts)
print 'True positives: ',perfs2[0]
print 'False positives: ',perfs2[1]
print 'True negatives: ',perfs2[2]
print 'False negatives: ',perfs2[3]
print 'accuracy: ',accuracy_score(y_test,y_pred_faceparts)
print 'precision: ',precision_score(y_test,y_pred_faceparts)
print 'recall: ',recall_score(y_test,y_pred_faceparts)

True positives:  435
False positives:  9
True negatives:  197
False negatives:  46
accuracy:  0.919941775837
precision:  0.97972972973
recall:  0.904365904366


In [130]:
#comment out if feature_importances_ exists, eg for decision tree based classifiers
#colnames=face_nonface_test.columns.values
#fi=clf.feature_importances_
#feature_importances=pd.Series(fi, index=colnames[2:])
#feature_importances

# Comparing results of the two face detection methods

In [102]:
face_nonface_test['y_pred_by_faceparts']=y_pred_faceparts
face_nonface_test['y_pred_by_facedetection']=y_pred_arr
face_nonface_test.head()

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist,y_pred_by_faceparts,y_pred_by_facedetection
0,BioID_0004.pgm,1,1,62.225397,62.225397,62.225397,62.225397,0.000000,4,79.275981,...,0.547079,1,70.830784,70.830784,70.830784,70.830784,2.434396,2.402659,1,1
1,BioID_0024.pgm,1,2,59.396970,57.982756,60.811183,59.396970,0.277792,4,69.286029,...,0.425732,1,79.404030,79.404030,79.404030,79.404030,0.626557,1.146314,1,1
2,BioID_0034.pgm,1,2,50.204581,46.669048,53.740115,50.204581,0.293798,4,70.872212,...,0.385505,1,80.262071,80.262071,80.262071,80.262071,1.098411,0.854272,1,1
3,BioID_0040.pgm,1,1,57.982756,57.982756,57.982756,57.982756,0.000000,5,81.907908,...,0.424366,1,73.573093,73.573093,73.573093,73.573093,0.255621,0.502901,1,1
4,BioID_0047.pgm,1,2,56.568542,55.154329,57.982756,56.568542,0.278423,4,75.588248,...,0.623973,1,69.971423,69.971423,69.971423,69.971423,0.977051,1.001046,1,1


In [122]:
#Examples for cases where face-detection-from-face-parts is better than „default” face detection
false=face_nonface_test.loc[(face_nonface_test['face'] == 1) & (face_nonface_test['y_pred_by_faceparts'] == 1) & (face_nonface_test['y_pred_by_facedetection']==0)]
false

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist,y_pred_by_faceparts,y_pred_by_facedetection
6,BioID_0054.pgm,1,1,73.539105,73.539105,73.539105,73.539105,0.000000,5,85.578074,...,0.591185,1,129.139460,129.139460,129.139460,129.139460,0.812463,0.886968,1,0
11,BioID_0104.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,3,67.285050,...,0.232465,1,66.370174,66.370174,66.370174,66.370174,0.000000,0.000000,1,0
21,BioID_0174.pgm,1,2,68.589358,63.639610,73.539105,68.589358,0.324394,3,77.746025,...,0.548859,1,75.802375,75.802375,75.802375,75.802375,1.222346,1.312690,1,0
22,BioID_0184.pgm,1,2,48.083261,48.083261,48.083261,48.083261,0.379550,4,76.488680,...,0.759493,1,67.742158,67.742158,67.742158,67.742158,1.022693,1.048326,1,0
25,BioID_0214.pgm,1,2,46.669048,45.254834,48.083261,46.669048,0.337483,2,82.319770,...,0.434282,0,0.000000,0.000000,0.000000,0.000000,1.412020,0.000000,1,0
60,BioID_0384.pgm,1,1,70.710678,70.710678,70.710678,70.710678,0.000000,1,64.140471,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.827647,0.000000,1,0
112,BioID_0564.pgm,1,2,60.104076,59.396970,60.811183,60.104076,0.332756,2,73.316508,...,0.456923,1,76.661594,76.661594,76.661594,76.661594,0.812702,1.174643,1,0
115,BioID_0594.pgm,1,1,35.355339,35.355339,35.355339,35.355339,0.000000,1,59.169249,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,2.306339,0.000000,1,0
161,BioID_0843.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,68.600292,...,0.000000,1,68.600292,68.600292,68.600292,68.600292,0.000000,0.000000,1,0
186,BioID_0949.pgm,1,2,64.346717,63.639610,65.053824,64.346717,0.303046,2,80.518666,...,0.481255,1,69.971423,69.971423,69.971423,69.971423,1.628692,1.307696,1,0


In [121]:
#Examples for cases where face-detection-from-face-parts is worse than „normal” face detection:
true=face_nonface_test.loc[(face_nonface_test['face'] == 1) & (face_nonface_test['y_pred_by_faceparts'] == 0) & (face_nonface_test['y_pred_by_facedetection']==1)]
true

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist,y_pred_by_faceparts,y_pred_by_facedetection
45,BioID_0324.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,2,76.917226,...,0.175513,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
46,BioID_0334.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,96.896852,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
48,BioID_0341.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,2,83.177586,...,0.135253,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
49,BioID_0342.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,67.742158,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
53,BioID_0346.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,85.234969,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
55,BioID_0348.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
56,BioID_0349.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,51.107729,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
57,BioID_0354.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,78.032045,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
58,BioID_0364.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,81.633327,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
62,BioID_0400.pgm,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,1,64.140471,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [119]:
#Examples for cases where face-detection-from-face-parts is better than „normal” face detection
false2=face_nonface_test.loc[(face_nonface_test['face'] == 0) & (face_nonface_test['y_pred_by_faceparts'] == 0) & (face_nonface_test['y_pred_by_facedetection']==1)]
false2

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist,y_pred_by_faceparts,y_pred_by_facedetection
509,0272.jpg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
563,AbrahamLincoln0079.jpeg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
567,alien2.jpg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,2,53.593942,...,0.653059,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
573,ElvisPresley0071.jpeg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,58.309519,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
581,LionelMessi0079.jpeg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
623,monkey_767.jpg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,122.449990,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
627,monkey_796.jpg,0,1,29.698485,29.698485,29.698485,29.698485,0.0,0,0.000000,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
632,QueenElizabethII0035.jpeg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,64.140471,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
641,TomCruise0057.jpeg,0,0,0.000000,0.000000,0.000000,0.000000,0.0,2,68.171202,...,0.333719,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
661,wolf_557.jpg,0,1,62.225397,62.225397,62.225397,62.225397,0.0,0,0.000000,...,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [120]:
#Examples for cases where face-detection-from-face-parts is worse than „normal” face detection:
true2=face_nonface_test.loc[(face_nonface_test['face'] == 0) & (face_nonface_test['y_pred_by_faceparts'] == 1) & (face_nonface_test['y_pred_by_facedetection']==0)]
true2

,filename,face,eyes_number_of,eyes_rect_mean,eyes_rect_min,eyes_rect_max,eyes_rect_median,eyes_placement_characteristip,mouth_number_of,mouth_rect_mean,...,mouth_placement_characteristip,mouth2_number_of,mouth2_rect_mean,mouth2_rect_min,mouth2_rect_max,mouth2_rect_median,mouth_eyes_dist,mouth2_eyes_dist,y_pred_by_faceparts,y_pred_by_facedetection
571,chimpanzee_1.jpg,0,1,65.053824,65.053824,65.053824,65.053824,0.000000,3,120.506924,...,0.615414,0,0.000000,0.000000,0.000000,0.000000,1.272085,0.000000,1,0
576,GeorgeClooney0074.jpeg,0,2,62.932504,59.396970,66.468037,62.932504,0.214516,2,73.573342,...,0.282032,1,81.633327,81.633327,81.633327,81.633327,0.608248,0.778471,1,0
584,monkey_127.jpg,0,2,89.802561,87.681241,91.923882,89.802561,0.256117,2,178.959059,...,0.178812,0,0.000000,0.000000,0.000000,0.000000,0.851887,0.000000,1,0
618,monkey_758.jpg,0,1,36.769553,36.769553,36.769553,36.769553,0.000000,2,145.774210,...,0.166353,0,0.000000,0.000000,0.000000,0.000000,1.538762,0.000000,1,0
622,monkey_766.jpg,0,1,41.012193,41.012193,41.012193,41.012193,0.000000,1,94.667840,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.458115,0.000000,1,0
655,wolf_47.jpg,0,2,51.618795,48.083261,55.154329,51.618795,0.261533,1,58.309519,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,3.253994,0.000000,1,0
